In [1]:
import re
import time
from tqdm import tqdm

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

#from selenium import webdriver

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from pymystem3 import Mystem
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import *

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
positive = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/positive.csv',
                       sep = ';',
                       header= None
                      )

In [4]:
negative = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/negative.csv',
                       sep = ';',
                       header= None
                      )

In [5]:
positive_text = pd.DataFrame(positive.iloc[:, 3])
negative_text = pd.DataFrame(negative.iloc[:, 3])
positive_text['label'] = [1] * positive_text.shape[0]
negative_text['label'] = [0] * negative_text.shape[0]

In [6]:
labeled_tweets = pd.concat([positive_text, negative_text])
labeled_tweets.index = range(labeled_tweets.shape[0])

labeled_tweets.columns = ['text', 'label']
labeled_tweets

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,0
226830,скучаю так :-( только @taaannyaaa вправляет мо...,0
226831,"Вот и в школу, в говно это идти уже надо(",0
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",0


## Очистка размеченного датасета *Им. начальника подотдела очистки тов. Шарикова П. П.*

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Оставим в тексте только кириллические символы
def clear_text(text):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    return " ".join(clear_text.split())


# напишем функцию удаляющую стоп-слова
def clean_stop_words(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return " ".join(text)

In [9]:
# загрузим список стоп-слов
stopwords = set(nltk_stopwords.words('russian'))
np.array(stopwords)

array({'или', 'ну', 'уже', 'мой', 'свою', 'была', 'почти', 'всегда', 'где', 'во', 'нее', 'нас', 'над', 'куда', 'этом', 'о', 'с', 'себе', 'да', 'разве', 'эту', 'нельзя', 'лучше', 'на', 'здесь', 'после', 'вот', 'быть', 'и', 'был', 'есть', 'про', 'чуть', 'она', 'он', 'было', 'под', 'опять', 'зачем', 'какая', 'им', 'его', 'к', 'вы', 'ее', 'ж', 'перед', 'от', 'чем', 'чтобы', 'надо', 'не', 'никогда', 'кто', 'ты', 'сейчас', 'за', 'три', 'больше', 'впрочем', 'иногда', 'этот', 'вас', 'то', 'когда', 'ней', 'такой', 'один', 'себя', 'ним', 'там', 'они', 'все', 'два', 'тоже', 'как', 'другой', 'ведь', 'тебя', 'же', 'тут', 'бы', 'без', 'теперь', 'того', 'ли', 'мы', 'по', 'что', 'до', 'них', 'если', 'но', 'потом', 'тем', 'мне', 'между', 'совсем', 'будет', 'только', 'хоть', 'вдруг', 'чего', 'этой', 'ей', 'даже', 'потому', 'какой', 'для', 'меня', 'том', 'со', 'много', 'ни', 'об', 'чтоб', 'сам', 'всего', 'так', 'может', 'этого', 'конечно', 'более', 'у', 'из', 'хорошо', 'тогда', 'еще', 'ему', 'их', 'я', '

In [10]:
# Протестируем работу функции очистки текста
text = labeled_tweets['text'][np.random.randint(labeled_tweets.shape[0])]
print(text)
print('=======================================')
print(clean_stop_words((clear_text(text)), stopwords))

У Мелешкевич такой смех дикий, что я просто не могу :))
мелешкевич смех дикий просто могу


## *Пока всё работало быстро. Посмотрим, что будет тут.*

In [11]:
%%time
labeled_tweets['text_clear'] = labeled_tweets['text']\
                              .apply(lambda x: clean_stop_words(clear_text(str(x)), stopwords))

CPU times: user 2.72 s, sys: 44 ms, total: 2.77 s
Wall time: 2.77 s


In [12]:
labeled_tweets = labeled_tweets[['text_clear', 'label']]
labeled_tweets

,text_clear,label
0,школота поверь самое общество профилирующий пр...,1
1,таки немного похож мальчик равно,1
2,идиотка испугалась,1
3,углу сидит погибает голода ещё порции взяли хо...,1
4,значит страшилка блин посмотрев части создастс...,1
...,...,...
226829,каждый хочет исправлять,0
226830,скучаю вправляет мозги равно скучаю,0
226831,школу говно это идти,0
226832,тауриэль грусти обнял,0


In [15]:
train, test = train_test_split(
  											labeled_tweets,
                        test_size = 0.2,
  											stratify = labeled_tweets['label'],
                        random_state = 12348,
                       )

# инцициализируем векторайзер и укажем размер n-грамм
counter_idf = TfidfVectorizer(ngram_range=(1,1))

# Получаем словарь и idf только из тренировочного набора данных
count_train = counter_idf.fit_transform(train['text_clear'])

# Применяем обученный векторайзер к тестовому набору данных
count_test = counter_idf.transform(test['text_clear'])

# Инициализируем модель с параметрами по умолчанию
model_lr = LogisticRegression(random_state = 12345,
                                    max_iter = 10000,
                                    n_jobs = -1)

# Подбираем веса для слов с помощь fit на тренировочном наборе данных
model_lr.fit(count_train, train['label'])

# Получаем прогноз модели на тестовом наборе данных
predict_count_proba = model_lr.predict_proba(count_test)

In [17]:
matrix = confusion_matrix(test['label'],
								# В качестве порогового значения вероятности
                # ниже которого объекты будут принадлежать
                # к классу негативных выберем 0.5
                 (predict_count_proba[:, 0] <  0.5).astype('int'),
                 normalize='true',
                )
matrix

array([[0.69676122, 0.30323878],
       [0.23805587, 0.76194413]])